### Lineare Regression auf dem Ames Housing Price Data Set von kaggle.com

##### Versionsgeschichte

    1.0 05.05.2023 Willi Hahn Initialversion
          Datenvorbereitung nach https://www.eamonfleming.com/projects/housing-regression.html
   


#### Bibliotheken einmalig installatieren
Einmalige Installation notwendiger Pakete wird in VL 3 beschrieben.
Verwendet werden hier die Bibliotheken pandas, numpy, matplotlib, seaborn, sklearn, imblearn
einmalig installieren, wenn ModuleNotFound error auftritt
dann Zelle als Zelltyp Code umwandeln und ausführen

!pip install pandas 
!pip install numpy 
!pip install sklearn 
!pip install matplotlib 
!pip install seaborn 
!pip install imbalanced-learn
!pip install math
!pip install itertools


In [ ]:
# notwendige Bibliotheken importieren und konfigurieren
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
_ = pd.set_option('display.max_columns', None) # damit mehr als 20 Spalten angezeigt werden.
pd.set_option('display.min_rows', 8) # damit nicht nur 10 Zeilen mit  ... dazwischen ausgegeben werden
pd.set_option('display.max_rows', 500) # damit nicht nur 10 Zeilen mit  ... dazwischen ausgegeben werden
import numpy as np
from collections import Counter
from timeit import default_timer as timer
from math import ceil
from itertools import zip_longest
import seaborn as sns #importing Seaborn's for plots
from sklearn import metrics as met 
import matplotlib.pyplot as plt #Plot Bibliothek
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score



In [ ]:
# Daten einlesen
path = 'c:/myBox/Projekte/FHDW/Kurs DAML/Daten/AmesHousing.csv' # für lokale Dateien
#path = 'https://github.com/Rechen47/FHDW.DAML/raw/main/AmesHousing.csv'

df = pd.read_csv(path, sep=',') 
data_org1 = df.copy() # Eine Datenkopie als Referenz behalten

df.info(verbose=True, show_counts=True)
print (df.describe())
df.head(20)

In [ ]:
#Datenverständnis
# Sind die Identifier eindeutig?
print(f"Spalte Order ist eindeutig: {pd.Series(df['Order']).is_unique}")
print(f"Spalte PID ist eindeutig  : {pd.Series(df['PID']).is_unique}")


In [ ]:
# Zielvariable SalePrice untersuchen
_ = plt.hist(df['SalePrice'],bins=30)
_ = plt.title('Histogram of House Sale Price')
_ = plt.xlabel('Sale Price');
plt.show()
# create boxplot of SalePrice
_ = plt.boxplot(df['SalePrice'],vert=False)
_ = plt.title('Boxplot of House Sale Price');
plt.show()

# log transform of the sale price and check the histogram and boxplot
_ = plt.hist(np.log(df['SalePrice']),bins=30)
_ = plt.title('Histogram of House Sale Price (Log transformed)')
_ = plt.xlabel('Sale Price (log transformed)');
plt.show()
# create boxplot of the log transformed SalePrice
_ = plt.boxplot(np.log(df['SalePrice']),vert=False)
_ = plt.title('Boxplot of House Sale Price (log transformed)');
plt.show()

# Boxplot SalePrice vs Neighborhood
_ = plt.figure(figsize=(20,10))
_ = sns.boxplot(x = 'Neighborhood',y = 'SalePrice',data=df,palette='viridis')
_ = plt.tight_layout()
_ = plt.title('SalePrice vs Neighborhood');

# Boxplot SalePrice vs Neighborhood
_ = plt.figure(figsize=(20,10))
_ = sns.boxplot(x = 'Roof Matl',y = 'SalePrice',data=df,palette='viridis')
_ = plt.title('SalePrice vs Roof Material');
plt.show()

# Boxplot SalePrice vs Kitchen Quality
_ = plt.figure(figsize=(20,10))
_ = sns.boxplot(x = 'Kitchen Qual',y = 'SalePrice',data=df,palette='viridis')
_ = plt.title('SalePrice vs Kitchen Quality');
plt.show()

In [ ]:
#Datenverständnis der Vororte / Nachbarschaft
print ("Median der Verkaufspreise je Vorort")
print (df.groupby('Neighborhood', as_index=True)['SalePrice'].median())
# hier wird eine neue Variable eingeführt
print ("Median der Verkaufspreise/SquareFoot je Vorort")
df['TotalLivingSF'] = df['Gr Liv Area'] + df['Total Bsmt SF'] - df['Low Qual Fin SF']
df.groupby('Neighborhood', as_index=True)['SalePrice'].mean()  /    df.groupby('Neighborhood', as_index=True)['TotalLivingSF'].mean()


# Plot Anzahl Verkäufe je Vorort
nbh_counts_df = df.Neighborhood.value_counts().to_frame('Anzahl').reset_index()
nbh_counts_df.rename(columns = {'index':'Vororte'}, inplace = True)
sns.set(rc={'figure.figsize':(20,5)})
plt.xticks(fontsize=12,rotation='vertical')
p=sns.barplot(x="Vororte", y="Anzahl", data=nbh_counts_df,palette="Greens",capsize=.2)
p.axes.set_title("\nAnzahl Verkäufe je Vorort\n",fontsize=30);

# Optional können Vororte mit wenigen Häuser zu einem Vorort "others" zusammengefasst werden. Hier werden weiter unten die Häuser von Landmark 
# und Grnhll entfernt.

In [ ]:
# Histogramme der numerischen und kategorialen Variablen
numerical_data = df.select_dtypes("number")
numerical_data.info()
numerical_data.hist(bins=20, figsize=(12, 22), edgecolor="black", layout=(10, 5))
plt.subplots_adjust(hspace=0.8, wspace=0.8)


string_data = df.select_dtypes(object)
string_data.info()

n_string_features = string_data.shape[1]
nrows, ncols = ceil(n_string_features / 4), 4
fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(14, 80))
for feature_name, ax in zip_longest(string_data, axs.ravel()):
    if feature_name is None:
        # do not show the axis
        ax.axis("off")
        continue
    string_data[feature_name].value_counts().plot.barh(ax=ax)
    ax.set_title(feature_name)
plt.subplots_adjust(hspace=0.2, wspace=0.8)



In [ ]:
# Liste der Pearson Korrelationskoeffzienten aller Variablen mit SalePrice (statt Heatmap)
# nach https://medium.com/@hjhuney/quantitative-data-exploration-for-regression-in-python-ames-housing-part-1-25879dd4cc4a
corr_list = sorted(df.corr(numeric_only=True).to_dict()['SalePrice'].items(), key=lambda x: x[1], reverse=True)
corr_list

In [ ]:
# Welche Spalten sind nicht nur in Bezug auf SalePrice sondern auch untereinander start korreliert?

def corrFilter(x: pd.DataFrame, bound: float):
    xCorr = x.corr(numeric_only = True)
    xFiltered = xCorr[((xCorr >= bound) | (xCorr <= -bound)) & (xCorr !=1.000)]
    xFlattened = xFiltered.unstack().sort_values().drop_duplicates()
    return xFlattened

corrFilter(df, .6)


In [ ]:
# fehlende Werte anzeigen
null_stats = pd.DataFrame(df.isnull().sum(), columns=['missing_value_count'])
null_stats.loc[:,'% of dataset'] = np.round(null_stats.loc[:,'missing_value_count'] / df.shape[0] * 100, 2)
null_stats.loc[null_stats.loc[:,'missing_value_count']>0,:].sort_values(by='missing_value_count', ascending=False)



In [ ]:
# Leerzeichen am Ende entfernen, hier nur Sale Type
df[df['Sale Type'].str.startswith(' ')]
df[df['Sale Type'].str.endswith(' ')]
#print(df['Sale Type'] + "#")
df['Sale Type'] = df['Sale Type'].str.strip()

In [ ]:
#One-hot encoding für alle nominalen Variables ohne Nullwerte
# except 'MS SubClass' since it is a combination of bldg.type, house.style, and year.built
#keep Neighborhood_org for later calculations
df['Neighborhood_org'] = df['Neighborhood']
df = pd.get_dummies(df, columns=['MS Zoning',
                                     'Street',
                                     'Land Contour',
                                     'Condition 1',
                                     'Condition 2',
                                     'House Style',
                                     'Lot Config',
                                     'Bldg Type',
                                     'Neighborhood',
                                     'Roof Style',
                                     'Roof Matl',
                                     'Exterior 1st',
                                     'Exterior 2nd', 
                                     'Foundation', 
                                     'Heating', 
                                     'Central Air',
                                     'Sale Condition',
                                     'Sale Type'], 
                      drop_first=True)
df.shape
df.info(verbose=True, show_counts=True)
df.describe()


In [ ]:
# Ordinale, nicht-numerische Spalten umwandeln in Ganzzahlen und dabei fehlende Werte behandeln

#Lot Shape will be assigned numeric values according to irregularity (0,1,2,3)
#IR2(76) and IR3 (16) combined into one level, because of same median sales price
df['Lot Shape'].value_counts()
df['Lot Shape'] = df['Lot Shape'].map({'Reg':0,'IR1':1,'IR2':2,'IR3':2})

#This seems somewhat negligible, but it may as well be included
df['Utilities'].value_counts()
df['Utilities'] = df['Utilities'].map({'AllPub':0,'NoSewr':1,'NoSeWa':2,'ELO':3})

#Gtl gentle ist der Nullpunkt, da es flat nicht gibt
df['Land Slope'].value_counts()
df['Land Slope'] = df['Land Slope'].map({'Gtl':0,'Mod':1,'Sev':2})

df['Exter Cond'].value_counts()
df['Kitchen Qual'].value_counts()
df['Exter Qual'].value_counts()
#These variables follow the same scale.  Notice the average condition is the majority in most cases. 
#The appropriate match here to me is -4, -1, 0, 1, 4.  The idea is that Excellent or Poor have a larger than proportional impact.
for i in ['Exter Qual','Exter Cond','Kitchen Qual']:
    df[i] = df[i].map({'Ex':4,'Gd':1,'TA':0,'Fa':-1,'Po':-4})


df['Heating QC'].value_counts()
df['Heating QC'] = df['Heating QC'].map({'Ex':2,'Gd':1,'TA':0,'Fa':-2,'Po':-4})

    
df['Electrical'].value_counts()
df['Electrical'] = df['Electrical'].map({'SBrkr':0,'FuseA':1,'FuseF':2, 'FuseP':3, 'Mix':1.5})
df['Electrical'] = df['Electrical'].fillna(1.5)
df['Functional'].value_counts()
df['Functional'] = df['Functional'].map({'Typ':0,'Min1':1,'Min2':2,'Mod':3,'Maj1':4,'Maj2':5,'Sev':6,'Sal':7})
df['Paved Drive'].value_counts()
df['Paved Drive'] = df['Paved Drive'].map({'N':0,'P':1,'Y':2})

In [ ]:
# Ordinale und 1 metrische Variablen zur Garage verbessern
missing_garage_data = df[df['Garage Finish'].isnull()==True]
missing_garage_data[['Garage Area','Garage Type','Garage Yr Blt','Garage Finish','Garage Cars','Garage Qual','Garage Cond']].head(10)

df['Garage Type'] = df['Garage Type'].fillna('None')
df['Garage Finish'] = df['Garage Finish'].fillna('None')
df['Garage Qual'] = df['Garage Qual'].fillna('None')
df['Garage Cond'] = df['Garage Cond'].fillna('None')
df[df['Garage Cars'].isnull()==True]    
df['Garage Cars'] = df['Garage Cars'].fillna(0)

#Nominal variable 'Garage Type' goes to one hot encoding
df = pd.get_dummies(df, columns=['Garage Type'], drop_first=True)

df['Garage Finish'] = df['Garage Finish'].map({'Fin':3,'RFn':2,'Unf':1,'None':0})

df['Garage Qual'] = df['Garage Qual'].map({'Ex':4,'Gd':3,'TA':2.5,'Fa':2,'Po':1.5,'None':0})
df['Garage Cond'] = df['Garage Cond'].map({'Ex':4,'Gd':3,'TA':2.5,'Fa':2,'Po':1.5,'None':0})

#Here I will impute the year house built for year garage built NaNs:
df['Garage Yr Blt'] = df['Garage Yr Blt'].fillna(df['Year Built'])

# 1 missing Area
df['Garage Area'] = df['Garage Area'].fillna(0)



In [ ]:
#Ordinale Variablen zu Fireplace, Pool and Fence quality verbessern
df['Fireplace Qu'] = df['Fireplace Qu'].fillna('None')
df['Fireplace Qu'] = df['Fireplace Qu'].map({'Ex':4,'Gd':3,'TA':2.5,'Fa':2,'Po':1.5,'None':0})
df['Pool QC'] = df['Pool QC'].fillna('None')
df['Pool QC'] = df['Pool QC'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'None':0})
df['Fence'] = df['Fence'].fillna('None')
df['Fence'] = df['Fence'].map({'GdPrv':3,'GdWo':2,'MnPrv':1,'MnWw':1,'None':0})

In [ ]:
# Ordinale Variablen zum Keller/Basement verbessern
missing_bsmt_data = df[df['Bsmt Exposure'].isnull()==True]
missing_bsmt_data[['Bsmt Exposure','Bsmt Qual','Bsmt Cond','BsmtFin Type 1','BsmtFin Type 2']].head(10)

df['Bsmt Qual'] = df['Bsmt Qual'].map({'Ex':4,'Gd':3,'TA':2.5,'Fa':2,'Po':1.5,'None':0})
df['Bsmt Cond'] = df['Bsmt Cond'].map({'Ex':4,'Gd':3,'TA':2.5,'Fa':2,'Po':1.5,'None':0})
df['BsmtFin Type 1'] = df['BsmtFin Type 1'].map({'GLQ':4,'ALQ':3.5,'BLQ':3,'Rec':2.5,'LwQ':2,'Unf':1.5,'None':0})
df['BsmtFin Type 2'] = df['BsmtFin Type 2'].map({'GLQ':4,'ALQ':3.5,'BLQ':3,'Rec':2.5,'LwQ':2,'Unf':1.5,'None':0})
df['Bsmt Exposure'] = df['Bsmt Exposure'].map({'Gd':3,'Av':2,'Mn':1,'No':0,'None':0})
df['Bsmt Qual'] = df['Bsmt Qual'].fillna(0)
df['BsmtFin Type 1'] = df['BsmtFin Type 1'].fillna(0)
df['BsmtFin Type 2'] = df['BsmtFin Type 2'].fillna(0)
df['Bsmt Exposure'] = df['Bsmt Exposure'].fillna(0)
df['Bsmt Cond'] = df['Bsmt Cond'].fillna(0)


In [ ]:
# fehlende Werte bei kontinuierlichen Variablen des basement verbessern
# In USA wird die Kellerfläche nicht zur Wohnfläche hinzu gerechnet.

df[df['BsmtFin SF 1'].isnull()]
df['BsmtFin SF 1'] = df['BsmtFin SF 1'].fillna(0)

# 2 Datensätzen mit fehlender Badangabe netfernen
df[df['Bsmt Full Bath'].isnull()]
for i in df[df['Bsmt Full Bath'].isnull()==True].loc[:,'Order']:
    df = df[df['Order']!=i]
df.shape  

In [ ]:
#Nominale Variablen mit und ohne Ordnung verbessern
#Convert existing Alley column into a dummy for Yes/No on Alley, and dummies for gravel alley and paved alley
df['Alley'] = df['Alley'].fillna(0)
df['Alley Grvl'] = df['Alley'].map(lambda x: 1 if x=='Grvl' else 0)
df['Alley Pave'] = df['Alley'].map(lambda x: 1 if x=='Pave' else 0)
df['Alley'] = df['Alley'].map(lambda x: 1 if x!=0 else 0)

#Miscellaneous dummy creation/imputation for nominal variables:
df = pd.get_dummies(df, columns=['Misc Feature'])

df['Mas Vnr Type'] = df['Mas Vnr Type'].fillna('None')
df = pd.get_dummies(df, columns=['Mas Vnr Type'])
df.drop('Mas Vnr Type_None',axis=1,inplace=True)

df['Mas Vnr Area'] = df['Mas Vnr Area'].fillna(0)

In [ ]:
#Lot Frontage Imputation
#For Lot Frontage, there are a large number of null values, and 0 is a senseless value for this.
#One approach would be to map in the average overall lot frontage mean for all the nans.
#However, a more accurate approach may be to map in the mean lot frontage for the neighborhood of each house.
neighborhood_means = df.groupby('Neighborhood_org')['Lot Frontage'].mean()
neighborhood_means = neighborhood_means.fillna(np.mean(neighborhood_means))
neighborhood_means.head()
df['Lot Frontage'] = df['Lot Frontage'].fillna(df['Neighborhood_org'])
df['Lot Frontage'] = df['Lot Frontage'].map(lambda x: neighborhood_means[x] if type(x)==str else x)






In [ ]:
# Variablen zum Dach verbessern
#Because 98.37% of roof.matl (roof material) is CompShg with the remaining spread across 7 categories, 
# we could transformed the data to Comp.Shg or not.


In [ ]:
# Variablen des Verkaufs verbessern.

df['Mo Sold org'] = df['Mo Sold']
df = pd.get_dummies(df, columns=['Mo Sold'], drop_first=False)

#Drop outliers of sale price below and above 1st and 99th percentile?

#Sollen Häuser mit Sale Type Family und Abnormal entfernt werden?


#Alternative für Data Leakage. Dann müssen die Behandlungen dieser Vars hier und oben entfernt werden.
#'Sale Type' ist kein Data Leakage, da zur Angebotszeit bekannt.
# df.drop(columns=['Mo Sold', 'Yr Sold', 'Sale Condition'], inplace=True, axis=1)



In [ ]:
#Neue Variablen
# TotalLivingSF bereits eingeführt in Zelle Datenverständnis der Vororte / Nachbarschaft
df['TotalLivingSF'] = df['Gr Liv Area'] + df['Total Bsmt SF'] - df['Low Qual Fin SF']

seasons = {
    'Spring':[3,4,5],
    'Summer':[6,7,8],
    'Fall':[9,10,11],
    'Winter':[12,1,2]
}
df['Season'] = df['Mo Sold org'].map(lambda x: [i for i in seasons if x in seasons[i]][0])
#Gets one hot encoding for month and season of sale
df = pd.get_dummies(df, columns=['Season'],drop_first=False)

df['totalSqFeet'] = df['Total Bsmt SF'] + df['1st Flr SF'] + df['2nd Flr SF']


#Four discrete variables relating to bathroom numbers: - bsmt.full.bath - number of full baths in the basement - bsmt.half.bath - number of half baths in the basement - full.bath - number of full baths above ground level - half.bath - number of half baths above ground level
#The majority of homes do not have any half baths or basement full baths. Both bsmt.full.bath and bsmt.half.bath have two missing values. Both missing values for the 2 variables correspond to row indices of 1342 and 1498. Observation 1342 is missing 10 predictors, all related to the basement, giving no indication of whether or not they have a basement. Observation 1498 has no basement as shown by other predictors, so I have given both missing variables a value of 0.
# combine all bathroom variables (above ground and basement full baths and half baths) into one total bathroom variable
df['totalBathroom'] = df['Full Bath']  + df['Bsmt Full Bath'] + 0.5 * (df['Half Bath'] + df['Bsmt Half Bath'] )

# Hausalter
df['houseAge'] = df['Yr Sold'] - df['Year Built']
df['GarageAge'] = df['Yr Sold'] - df['Garage Yr Blt']
df['RemodelAge'] = df['Yr Sold'] - df['Year Remod/Add']
df['isreModeled'] = np.where(df['Year Remod/Add']  == df['Year Built'], 1, 0)
df['isNew'] = np.where(df['Yr Sold'] == df['Year Built'], 1, 0)


In [ ]:
# Widersprüche in den Variablen
# Verkauft bevor gebaut, remodeld?
df.loc[df['Yr Sold'] < df['Year Built']]
df.loc[df['Yr Sold'] < df['Year Remod/Add']]
df.loc[df['Year Built'] > df['Year Remod/Add']]
df.loc[df['Year Remod/Add'] < df['Year Built'], 'Year Remod/Add'] = df['Year Built']
df.loc[df['Year Remod/Add'] > df['Yr Sold'], 'Year Remod/Add'] = df['Yr Sold']

# Anzahl Zimmer
# Anzahl und Größe von Garagen, ungültiges Baujahr
df.loc[df['Yr Sold'] < df['Garage Yr Blt']]
df.loc[df['Yr Sold'] < df['Garage Yr Blt'], 'Garage Yr Blt'] = df['Year Built']

df.loc[df['Garage Yr Blt'] == 0]

# Mansarde vorhanden aber 0 qm?
# Keller vorhanden aber 0qm?
# Gibt es misc val ohne misc feature oder umgekehrt?


In [ ]:
df.info(verbose=True)

In [ ]:
# Spalten und Datensätze entfernen, wenn keine oder zuwenig Information, Ausreißer, Data Leakage oder Kolinearität

# Eindeutige Identifizierer entfernen
df.drop(columns=['Order'], inplace=True, axis=1)
df.drop(columns=['PID'], inplace=True, axis=1)

# 3 Datensätze entfernen, die nach Dokumentation Ausreißer sind
df.drop(index=df.loc[df.loc[:,'Gr Liv Area'] > 4000].index, inplace=True)

df.drop('Neighborhood_org',axis=1,inplace=True)

# Garage Cars ist stark korreliert mit Garage Area
df.drop('Garage Cars',axis=1,inplace=True)


df.drop('Year Remod/Add',axis=1,inplace=True)
df.drop('Year Built',axis=1,inplace=True)
df.drop('Yr Sold',axis=1,inplace=True)
df.drop('Mo Sold org',axis=1,inplace=True)

# drop the ms.subclass variable as it is a combination of bldg.type, house.style, and year.built.
df.drop('MS SubClass',axis=1,inplace=True)

df.drop(['Condition 2_Feedr',
           'Condition 2_RRAe',
           'Condition 2_RRAn',
           'Condition 2_RRNn',
           'Exterior 1st_CBlock',
           'Exterior 1st_ImStucc',
           'Exterior 1st_Stone',
           'Exterior 2nd_Stone',
           'Heating_Wall',
           'Misc Feature_TenC',
           'Misc Feature_Elev',
           'Roof Matl_CompShg',
           'Roof Matl_Membran'],
           axis=1,inplace=True)


# drop 3 observations from neighborhood = "GrnHill" "Landmrk", because they are too less
df.loc[df['Neighborhood_Landmrk'] == 1]
df.loc[df['Neighborhood_GrnHill'] == 1]
df.drop('Neighborhood_Landmrk',axis=1,inplace=True)
df.drop('Neighborhood_GrnHill',axis=1,inplace=True)


In [ ]:
# check
df.info(verbose=True, show_counts=True)
df.isnull().sum().sum()
# fehlende Werte anzeigen
null_stats = pd.DataFrame(df.isnull().sum(), columns=['missing_value_count'])
null_stats.loc[:,'% of dataset'] = np.round(null_stats.loc[:,'missing_value_count'] / df.shape[0] * 100, 2)
null_stats.loc[null_stats.loc[:,'missing_value_count']>0,:].sort_values(by='missing_value_count', ascending=False)


In [ ]:
print(df.columns.tolist())

df2 = df[['houseAge', 'SalePrice']].copy()

#df2 = df[['houseAge', 'totalSqFeet',  
#    'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr'
#    , 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_Greens', 'Neighborhood_IDOTRR'
#    , 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes'
#    , 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer'
#    , 'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker'
#    ,'SalePrice']].copy()
#
#df2 = df[['houseAge', 'totalSqFeet',  
#    'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr'
#    , 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_Greens', 'Neighborhood_IDOTRR'
#    , 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes'
#    , 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer'
#    , 'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker'
#    , 'Lot Frontage', 'Lot Area', 'Alley', 'Lot Shape'
#    , 'Utilities', 'Land Slope', 'Overall Qual', 'Overall Cond', 'Mas Vnr Area', 'Exter Qual', 'Exter Cond'
#    , 'Total Bsmt SF', 'Bsmt Qual', 'Bsmt Cond'
#    ,'SalePrice']].copy()

#df2 = df[['houseAge', 'GarageAge', 'RemodelAge'
#    , 'totalSqFeet', 'Lot Area'  
#    , 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr'
#    , 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_Greens', 'Neighborhood_IDOTRR'
#    , 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes'
#    , 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer'
#    , 'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker'
#    , 'Lot Frontage', 'Lot Area', 'Alley', 'Lot Shape'
#    , 'Utilities', 'Land Slope', 'Overall Qual', 'Overall Cond', 'Mas Vnr Area', 'Exter Qual', 'Exter Cond'
#    ,  'Fireplaces'
#    , 'Total Bsmt SF', 'Bsmt Qual', 'Bsmt Cond'
#    , 'totalBathroom'
#    , 'Roof Style_Gable', 'Roof Style_Gambrel', 'Roof Style_Hip', 'Roof Style_Mansard', 'Roof Style_Shed'
#    , 'Roof Matl_Metal', 'Roof Matl_Roll', 'Roof Matl_Tar&Grv', 'Roof Matl_WdShake', 'Roof Matl_WdShngl'
#    , 'Garage Area', 'Garage Qual'
#    , 'SalePrice']].copy()
#
## di ersten ca 18 Zeilen von df.columns -> R2 0,90, MAE 16906
#df2 = df[['Lot Frontage', 'Lot Area', 'Alley', 'Lot Shape', 'Utilities', 'Land Slope', 'Overall Qual', 'Overall Cond'
#          , 'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1'
#          , 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Heating QC'
#          , 'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath'
#          , 'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual'
#          , 'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu', 'Garage Yr Blt', 'Garage Finish'
#          , 'Garage Area', 'Garage Qual', 'Garage Cond', 'Paved Drive', 'Wood Deck SF', 'Open Porch SF'
#          , 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Pool QC', 'Fence', 'Misc Val'
#          , 'TotalLivingSF', 'MS Zoning_C (all)', 'MS Zoning_FV', 'MS Zoning_I (all)', 'MS Zoning_RH', 'MS Zoning_RL'
#          , 'MS Zoning_RM', 'Street_Pave', 'Land Contour_HLS', 'Land Contour_Low', 'Land Contour_Lvl', 'Condition 1_Feedr'
#          , 'Condition 1_Norm', 'Condition 1_PosA', 'Condition 1_PosN', 'Condition 1_RRAe', 'Condition 1_RRAn', 'Condition 1_RRNe'
#    , 'SalePrice']].copy()
#
## die ersten 22 Zeilen von df.columns -> R2 0,924, MAE 15769
#df2 = df[['Lot Frontage', 'Lot Area', 'Alley', 'Lot Shape', 'Utilities', 'Land Slope', 'Overall Qual', 'Overall Cond'
#          , 'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1'
#          , 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Heating QC'
#          , 'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath'
#          , 'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual'
#          , 'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu', 'Garage Yr Blt', 'Garage Finish'
#          , 'Garage Area', 'Garage Qual', 'Garage Cond', 'Paved Drive', 'Wood Deck SF', 'Open Porch SF'
#          , 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Pool QC', 'Fence', 'Misc Val'
#          , 'TotalLivingSF', 'MS Zoning_C (all)', 'MS Zoning_FV', 'MS Zoning_I (all)', 'MS Zoning_RH', 'MS Zoning_RL'
#          , 'MS Zoning_RM', 'Street_Pave', 'Land Contour_HLS', 'Land Contour_Low', 'Land Contour_Lvl', 'Condition 1_Feedr'
#          , 'Condition 1_Norm', 'Condition 1_PosA', 'Condition 1_PosN', 'Condition 1_RRAe', 'Condition 1_RRAn', 'Condition 1_RRNe'
#          , 'Condition 1_RRNn', 'Condition 2_Norm', 'Condition 2_PosA', 'Condition 2_PosN', 'House Style_1.5Unf', 'House Style_1Story', 'House Style_2.5Fin', 'House Style_2.5Unf', 'House Style_2Story', 'House Style_SFoyer', 'House Style_SLvl', 'Lot Config_CulDSac', 'Lot Config_FR2', 'Lot Config_FR3', 'Lot Config_Inside', 'Bldg Type_2fmCon', 'Bldg Type_Duplex', 'Bldg Type_Twnhs', 'Bldg Type_TwnhsE', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_Greens', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker', 'Roof Style_Gable', 'Roof Style_Gambrel', 'Roof Style_Hip', 'Roof Style_Mansard', 'Roof Style_Shed', 'Roof Matl_Metal', 'Roof Matl_Roll', 'Roof Matl_Tar&Grv', 'Roof Matl_WdShake', 'Roof Matl_WdShngl', 'Exterior 1st_AsphShn', 'Exterior 1st_BrkComm', 'Exterior 1st_BrkFace', 'Exterior 1st_CemntBd', 'Exterior 1st_HdBoard', 'Exterior 1st_MetalSd'
#          , 'Exterior 1st_Plywood'
#    , 'SalePrice']].copy()

# Test auf Erweiterung mit weiteren Felder, weil MAE dann zu groß wird
#df2 = df[['Lot Frontage', 'Lot Area', 'Alley', 'Lot Shape', 'Utilities', 'Land Slope', 'Overall Qual', 'Overall Cond'
#          , 'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1'
#          , 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Heating QC'
#          , 'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath'
#          , 'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual'
#          , 'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu', 'Garage Yr Blt', 'Garage Finish'
#          , 'Garage Area', 'Garage Qual', 'Garage Cond', 'Paved Drive', 'Wood Deck SF', 'Open Porch SF'
#          , 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Pool QC', 'Fence', 'Misc Val'
#          , 'TotalLivingSF', 'MS Zoning_C (all)', 'MS Zoning_FV', 'MS Zoning_I (all)', 'MS Zoning_RH', 'MS Zoning_RL'
#          , 'MS Zoning_RM', 'Street_Pave', 'Land Contour_HLS', 'Land Contour_Low', 'Land Contour_Lvl', 'Condition 1_Feedr'
#          , 'Condition 1_Norm', 'Condition 1_PosA', 'Condition 1_PosN', 'Condition 1_RRAe', 'Condition 1_RRAn', 'Condition 1_RRNe'
#          , 'Condition 1_RRNn', 'Condition 2_Norm', 'Condition 2_PosA', 'Condition 2_PosN', 'House Style_1.5Unf', 'House Style_1Story', 'House Style_2.5Fin', 'House Style_2.5Unf', 'House Style_2Story', 'House Style_SFoyer', 'House Style_SLvl', 'Lot Config_CulDSac', 'Lot Config_FR2', 'Lot Config_FR3', 'Lot Config_Inside', 'Bldg Type_2fmCon', 'Bldg Type_Duplex', 'Bldg Type_Twnhs', 'Bldg Type_TwnhsE', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_Greens', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker', 'Roof Style_Gable', 'Roof Style_Gambrel', 'Roof Style_Hip', 'Roof Style_Mansard', 'Roof Style_Shed', 'Roof Matl_Metal', 'Roof Matl_Roll', 'Roof Matl_Tar&Grv', 'Roof Matl_WdShake', 'Roof Matl_WdShngl', 'Exterior 1st_AsphShn', 'Exterior 1st_BrkComm', 'Exterior 1st_BrkFace', 'Exterior 1st_CemntBd', 'Exterior 1st_HdBoard', 'Exterior 1st_MetalSd'
#          , 'Exterior 1st_Plywood'
#          ,  'Exterior 1st_PreCast', 'Exterior 1st_Stucco', 'Exterior 1st_VinylSd', 'Exterior 1st_Wd Sdng', 'Exterior 1st_WdShing'
#          , 'Exterior 2nd_AsphShn', 'Exterior 2nd_Brk Cmn', 'Exterior 2nd_BrkFace', 'Exterior 2nd_CBlock', 'Exterior 2nd_CmentBd'
#          , 'Exterior 2nd_HdBoard', 'Exterior 2nd_ImStucc', 'Exterior 2nd_MetalSd', 'Exterior 2nd_Other', 'Exterior 2nd_Plywood'
## MAE 39000          , 'Exterior 2nd_PreCast', 'Exterior 2nd_Stucco', 'Exterior 2nd_VinylSd', 'Exterior 2nd_Wd Sdng', 'Exterior 2nd_Wd Shng'
## MAE 50000         , 'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab', 'Foundation_Stone', 'Foundation_Wood', 'Heating_GasA'
## MAE 360000          , 'Heating_GasW', 'Heating_Grav', 'Heating_OthW', 'Central Air_Y', 'Sale Condition_AdjLand', 'Sale Condition_Alloca'
##          , 'Sale Condition_Family', 'Sale Condition_Normal', 'Sale Condition_Partial', 'Sale Type_CWD', 'Sale Type_Con'
##          , 'Sale Type_ConLD', 'Sale Type_ConLI'
#          , 'Sale Type_ConLw'
#    , 'SalePrice']].copy()
#

#df2=df

#df2 = df[[ 'Lot Frontage', 'Lot Area', 'Alley', 'Lot Shape', 'Utilities', 'Land Slope', 'Overall Qual', 'Overall Cond',
#    'totalSqFeet', 'totalBathroom', 'houseAge', 'GarageAge', 'RemodelAge', 'isreModeled', 'isNew','SalePrice']].copy()

In [ ]:
# Trainings- und Testdaten aufteilen

TESTANTEIL = 0.25 # Split der Test- und Trainingsdaten
# Trennung von unabhängigen Variablen und abhängiger Zielvariable
y = df2['SalePrice']
x = df2.drop(['SalePrice'], axis = 1)
#x.head().T
#y.head().T
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TESTANTEIL, random_state=4711)

# und prüfen durch ansehen
print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)


In [ ]:
## Datenvorbereitung : Variablen skalieren
SCALER = StandardScaler()
#SCALER = MinMaxScaler()

x_train = SCALER.fit_transform(x_train)
x_test = SCALER.transform(x_test)
#y_test = SCALER.transform(y_test)
#y_train = SCALER.transform(y_train)
#
x_test

In [ ]:
#Lineare Regression berechnen
regressor = LinearRegression()
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

# COMPARING TEST DATA AND PREDICTED DATA
comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":y_pred})
print('Actual test data vs predicted: \n', comparison_df)

# EVALUATING MODEL METRICS
print('MAE:', mean_absolute_error(y_test,y_pred))
print("MSE",mean_squared_error(y_test,y_pred))
print("RMSE",np.sqrt(mean_squared_error(y_test,y_pred)))
r2 = r2_score(y_test,y_pred)
print('Model Score R2: ', r2)
print('\n\nCoefficients: ', regressor.coef_)
print('Intercept: ',regressor.intercept_)




In [ ]:
# Detailanalysen für Verkaufs- oder Renovierungsignalen

# Entscheidungshilfen für Zeitpunkt von Kauf oder Verkauf
# siehe Zelle zu Neue Variablen, Season

# Bilden Sie im Folgenden künstliche Datensätzen und berechnete mit dem besten Modell Variationen einer untersuchten Variablen.

# Entscheidungshilfe für mögliche Umbauten zur Steigerung des Verkaufpreises
# Einfluss einer zusätzlichen Küche
# Einfluss eines zusätzlichen Bades
# Einfluss eines zusätzlichen Schuppens
# Einfluss 1- 4 zusätzlichen Garagen
